# Logistic Regression

In [2]:
# Data Manipulation
import numpy as np
import pandas as pd
from datetime import datetime

# Plotting graphs
import matplotlib.pyplot as plt

# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import normalize

from ta import add_all_ta_features #pip install --upgrade ta https://github.com/bukosabino/ta https://medium.datadriveninvestor.com/predicting-the-stock-market-with-python-bba3cf4c56ef
from fastai.tabular.all import add_datepart #pip install fastai https://docs.fast.ai/tabular.core.html https://www.analyticsvidhya.com/blog/2018/10/predicting-stock-price-machine-learningnd-deep-learning-techniques-python/

### (Part1) Baseline Model without technical features

In this part, we will run a logistic regression with 3 features (High, Low, Open), which are scraped from Yahoo Finance. We will use Apple Stock Price data from 2021-12-29 to 2021-4-28 (4month). 

In [3]:
df = pd.read_csv('data/AAPL_data.csv')

In [4]:
df.dropna()
df = df[df['Date'] >= '2021-12-29']
df

,Date,High,Low,Open,Close,Volume,Adj Close
1176,2021-12-29,180.630005,178.139999,179.330002,179.380005,62348900.0,179.151749
1177,2021-12-30,180.570007,178.089996,179.470001,178.199997,59773000.0,177.973251
1178,2021-12-31,179.229996,177.259995,178.089996,177.570007,64062300.0,177.344055
1179,2022-01-03,182.880005,177.710007,177.830002,182.009995,104487900.0,181.778397
1180,2022-01-04,182.940002,179.119995,182.630005,179.699997,99310400.0,179.471344
...,...,...,...,...,...,...,...
1255,2022-04-22,167.869995,161.500000,166.460007,161.789993,84775200.0,161.789993
1256,2022-04-25,163.169998,158.460007,161.119995,162.880005,96046400.0,162.880005
1257,2022-04-26,162.339996,156.720001,162.250000,156.800003,95623200.0,156.800003
1258,2022-04-27,159.789993,155.380005,155.910004,156.570007,88063200.0,156.570007


**Add classification variable**  
- y = 1 if the next day's closing price is higher than today's closing price  
- y = -1 if the next day's closing price is lower than today's closing price

In [5]:
X = df[['High', 'Low', 'Open']]
y = np.where(df['Close'].shift(-1) > df['Close'], 1, -1)

**Split data to train and test set**

In [6]:
split = int(0.8 * len(df))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

In [7]:
print(len(X_train), len(X_test))

67 17


**Logistic Regression**  
- train the model with X_train and y_train

In [8]:
model = LogisticRegression(max_iter=float('inf'))
model = model.fit(X_train, y_train)

In [9]:
pd.DataFrame(zip(X.columns, np.transpose(model.coef_)))

,0,1
0,High,[0.26218930818524544]
1,Low,[0.15304821604498062]
2,Open,[-0.47929093522700755]


In [10]:
y_predict = model.predict(X_test)

**Evaluate the model**

In [11]:
metrics.confusion_matrix(y_test, y_predict)

array([[7, 4],
       [4, 2]], dtype=int64)

In [12]:
print(metrics.classification_report(y_test, y_predict))

              precision    recall  f1-score   support

          -1       0.64      0.64      0.64        11
           1       0.33      0.33      0.33         6

    accuracy                           0.53        17
   macro avg       0.48      0.48      0.48        17
weighted avg       0.53      0.53      0.53        17



### Conclusion: Accuracy of this model is 0.53.

In [13]:
model.score(X_test, y_test)

0.5294117647058824

### (Part2) Baseline Model with selected technical features

In this part, we will run a logistic regression with selected technical features. We will continue to use Apple Stock Price data from 2021-12-29 to 2021-4-28 (4month), but previous data are also used to calculate technical features. 

In [14]:
df = pd.read_csv('data/AAPL_data.csv')
df.dropna()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2017-04-28,36.075001,35.817501,36.022499,35.912498,83441600.0,33.907143
1,2017-05-01,36.799999,36.240002,36.275002,36.645000,134411600.0,34.598736
2,2017-05-02,37.022499,36.709999,36.884998,36.877499,181408800.0,34.818253
3,2017-05-03,36.872501,36.067501,36.397499,36.764999,182788000.0,34.712040
4,2017-05-04,36.785000,36.452499,36.630001,36.632500,93487600.0,34.586937
...,...,...,...,...,...,...,...
1255,2022-04-22,167.869995,161.500000,166.460007,161.789993,84775200.0,161.789993
1256,2022-04-25,163.169998,158.460007,161.119995,162.880005,96046400.0,162.880005
1257,2022-04-26,162.339996,156.720001,162.250000,156.800003,95623200.0,156.800003
1258,2022-04-27,159.789993,155.380005,155.910004,156.570007,88063200.0,156.570007


**Add date features**

Produced features : `Year`, `Month`, `Week`, `Day`, `Dayofweek`, `Dayofyear`, `Is_month_end`, `Is_month_start`, `Is_quarter_end`, `Is_quarter_start`, `Is_year_end`, `Is_year_start`

In [15]:

df["Date"]=pd.to_datetime(df.Date, format="%Y-%m-%d")
df.index=df['Date']
data = df.sort_index(ascending=True, axis=0)
add_datepart(df, 'Date', drop=False)
df.drop('Elapsed', axis=1, inplace=True)
df

,Date,High,Low,Open,Close,Volume,Adj Close,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start
Date,,,,,,,,,,,,,,,,,,,
2017-04-28,2017-04-28,36.075001,35.817501,36.022499,35.912498,83441600.0,33.907143,2017,4,17,28,4,118,False,False,False,False,False,False
2017-05-01,2017-05-01,36.799999,36.240002,36.275002,36.645000,134411600.0,34.598736,2017,5,18,1,0,121,False,True,False,False,False,False
2017-05-02,2017-05-02,37.022499,36.709999,36.884998,36.877499,181408800.0,34.818253,2017,5,18,2,1,122,False,False,False,False,False,False
2017-05-03,2017-05-03,36.872501,36.067501,36.397499,36.764999,182788000.0,34.712040,2017,5,18,3,2,123,False,False,False,False,False,False
2017-05-04,2017-05-04,36.785000,36.452499,36.630001,36.632500,93487600.0,34.586937,2017,5,18,4,3,124,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-22,2022-04-22,167.869995,161.500000,166.460007,161.789993,84775200.0,161.789993,2022,4,16,22,4,112,False,False,False,False,False,False
2022-04-25,2022-04-25,163.169998,158.460007,161.119995,162.880005,96046400.0,162.880005,2022,4,17,25,0,115,False,False,False,False,False,False
2022-04-26,2022-04-26,162.339996,156.720001,162.250000,156.800003,95623200.0,156.800003,2022,4,17,26,1,116,False,False,False,False,False,False


**Add technical features**

Produced features : all features from https://github.com/bukosabino/ta 

In [16]:
df = add_all_ta_features(
    df, high="High", low="Low", open="Open", close="Close", volume="Volume")

C:\Users\jwnic\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\jwnic\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


**Limit number of technical features based on previous literature**

In [17]:
df = df[df['Date'] >= '2021-12-29']
selected_features = ['trend_sma_fast', 'trend_ema_fast', 'momentum_stoch_rsi_k', 'momentum_stoch_rsi_d', 'momentum_rsi', \
                    'trend_macd', 'momentum_wr', 'volume_adi', 'momentum_roc', 'volume_obv', \
                    'volatility_bbh', 'volatility_bbl']
basic_features = ['High', 'Low', 'Open', 'Volume', 'Year', 'Month', 'Week', 'Day', 'Dayofweek']

X = df[selected_features + basic_features]
y = np.where(df['Close'].shift(-1) > df['Close'], 1, -1)

**Split data to train and test set**

In [18]:
split = int(0.8 * len(df))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

**Logistic Regression**  

In [19]:
model = LogisticRegression(max_iter=float('inf'))
model = model.fit(X_train, y_train)

In [20]:
pd.DataFrame(zip(X.columns, np.transpose(model.coef_)))

,0,1
0,trend_sma_fast,[-1.578970083714804e-15]
1,trend_ema_fast,[-1.7173939408291484e-15]
2,momentum_stoch_rsi_k,[7.654680588175565e-18]
3,momentum_stoch_rsi_d,[-5.504585429676596e-18]
4,momentum_rsi,[1.9292212127689274e-15]
5,trend_macd,[-2.1784100081387899e-16]
6,momentum_wr,[3.2832165556283935e-15]
7,volume_adi,[5.164356559687822e-10]
8,momentum_roc,[1.140994102965881e-15]
9,volume_obv,[-1.1697744284653211e-09]


**Evaluate the model**

In [21]:
y_predict = model.predict(X_test)
metrics.confusion_matrix(y_test, y_predict)
print(metrics.classification_report(y_test, y_predict))

              precision    recall  f1-score   support

          -1       0.70      0.64      0.67        11
           1       0.43      0.50      0.46         6

    accuracy                           0.59        17
   macro avg       0.56      0.57      0.56        17
weighted avg       0.60      0.59      0.59        17



### Conclusion: Accuracy of this model is 0.59.

The accuracy improved by including more features. 

In [22]:
model.score(X_test, y_test)

0.5882352941176471

### (Part3) Baseline Model with full set of features

In this part, we will run a logistic regression with full set of 100 features. We will continue to use Apple Stock Price data from 2021-12-29 to 2021-4-28 (4month), but previous data are also used to calculate technical features. 

**Include all of technical features except for the following**

In [23]:
X = df.drop(['Close', 'trend_psar_down', 'trend_psar_up', 'Date', 'Adj Close'], axis=1)
X.isnull().sum().sort_values()

High                       0
trend_visual_ichimoku_a    0
trend_cci                  0
trend_adx_neg              0
trend_adx_pos              0
                          ..
volatility_bbl             0
volatility_bbh             0
volatility_bbm             0
volatility_kcw             0
others_cr                  0
Length: 100, dtype: int64

**Split data to train and test set**

In [24]:
split = int(0.8 * len(df))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

**Logistic Regression**  

In [25]:

model = LogisticRegression(max_iter=float('inf'))
model = model.fit(X_train, y_train)
pd.DataFrame(zip(X.columns, np.transpose(model.coef_)))

,0,1
0,High,[-1.845109709345391e-15]
1,Low,[-2.888933849815397e-15]
2,Open,[-4.01931346737426e-15]
3,Volume,[1.2923619140371603e-08]
4,Year,[1.6072889172661235e-14]
...,...,...
95,momentum_pvo_hist,[1.1274957214997917e-15]
96,momentum_kama,[-1.2830052121191127e-15]
97,others_dr,[1.646312121431329e-15]
98,others_dlr,[1.6405120295601796e-15]


**Evaluate the model**

In [26]:
y_predict = model.predict(X_test)
metrics.confusion_matrix(y_test, y_predict)
print(metrics.classification_report(y_test, y_predict))

              precision    recall  f1-score   support

          -1       0.58      0.64      0.61        11
           1       0.20      0.17      0.18         6

    accuracy                           0.47        17
   macro avg       0.39      0.40      0.40        17
weighted avg       0.45      0.47      0.46        17



### Conclusion: Accuracy of this model is 0.47.

The accuracy dropped because we included too many features (over-fitting problem). 

In [27]:
model.score(X_test, y_test)

0.47058823529411764

### Normalizing Features

We can observe that volume had a coefficient that was orders of magnitude larger than other coefficients in the model. This leads us to try normalizing the features to see if this improves the models' predictive power.

In [41]:
df = pd.read_csv('data/AAPL_data.csv')
df.dropna()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2017-04-28,36.075001,35.817501,36.022499,35.912498,83441600.0,33.907143
1,2017-05-01,36.799999,36.240002,36.275002,36.645000,134411600.0,34.598736
2,2017-05-02,37.022499,36.709999,36.884998,36.877499,181408800.0,34.818253
3,2017-05-03,36.872501,36.067501,36.397499,36.764999,182788000.0,34.712040
4,2017-05-04,36.785000,36.452499,36.630001,36.632500,93487600.0,34.586937
...,...,...,...,...,...,...,...
1255,2022-04-22,167.869995,161.500000,166.460007,161.789993,84775200.0,161.789993
1256,2022-04-25,163.169998,158.460007,161.119995,162.880005,96046400.0,162.880005
1257,2022-04-26,162.339996,156.720001,162.250000,156.800003,95623200.0,156.800003
1258,2022-04-27,159.789993,155.380005,155.910004,156.570007,88063200.0,156.570007


In [42]:
df["Date"]=pd.to_datetime(df.Date, format="%Y-%m-%d")
df.index=df['Date']
data = df.sort_index(ascending=True, axis=0)
add_datepart(df, 'Date', drop=False)
df.drop('Elapsed', axis=1, inplace=True)
df

,Date,High,Low,Open,Close,Volume,Adj Close,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start
Date,,,,,,,,,,,,,,,,,,,
2017-04-28,2017-04-28,36.075001,35.817501,36.022499,35.912498,83441600.0,33.907143,2017,4,17,28,4,118,False,False,False,False,False,False
2017-05-01,2017-05-01,36.799999,36.240002,36.275002,36.645000,134411600.0,34.598736,2017,5,18,1,0,121,False,True,False,False,False,False
2017-05-02,2017-05-02,37.022499,36.709999,36.884998,36.877499,181408800.0,34.818253,2017,5,18,2,1,122,False,False,False,False,False,False
2017-05-03,2017-05-03,36.872501,36.067501,36.397499,36.764999,182788000.0,34.712040,2017,5,18,3,2,123,False,False,False,False,False,False
2017-05-04,2017-05-04,36.785000,36.452499,36.630001,36.632500,93487600.0,34.586937,2017,5,18,4,3,124,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-22,2022-04-22,167.869995,161.500000,166.460007,161.789993,84775200.0,161.789993,2022,4,16,22,4,112,False,False,False,False,False,False
2022-04-25,2022-04-25,163.169998,158.460007,161.119995,162.880005,96046400.0,162.880005,2022,4,17,25,0,115,False,False,False,False,False,False
2022-04-26,2022-04-26,162.339996,156.720001,162.250000,156.800003,95623200.0,156.800003,2022,4,17,26,1,116,False,False,False,False,False,False


In [43]:
df = add_all_ta_features(
    df, high="High", low="Low", open="Open", close="Close", volume="Volume")

C:\Users\jwnic\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\jwnic\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [44]:
df = df[df['Date'] >= '2021-12-29']
selected_features = ['trend_sma_fast', 'trend_ema_fast', 'momentum_stoch_rsi_k', 'momentum_stoch_rsi_d', 'momentum_rsi', \
                    'trend_macd', 'momentum_wr', 'volume_adi', 'momentum_roc', 'volume_obv', \
                    'volatility_bbh', 'volatility_bbl']
basic_features = ['High', 'Low', 'Open', 'Volume', 'Year', 'Month', 'Week', 'Day', 'Dayofweek']
X_df = df[selected_features + basic_features]
X = normalize(X_df.values)
y = np.where(df['Close'].shift(-1) > df['Close'], 1, -1)
split = int(0.8 * len(df))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]
model = LogisticRegression(max_iter=float('inf'))
model = model.fit(X_train, y_train)
pd.DataFrame(zip(X_df.columns, np.transpose(model.coef_)))

,0,1
0,trend_sma_fast,[-2.8674269772994965e-09]
1,trend_ema_fast,[-2.7736171329990864e-09]
2,momentum_stoch_rsi_k,[1.2543362550511096e-10]
3,momentum_stoch_rsi_d,[1.0508309595167229e-10]
4,momentum_rsi,[7.357387507982627e-10]
5,trend_macd,[-1.6253090720471196e-09]
6,momentum_wr,[5.576301585612237e-09]
7,volume_adi,[0.16121919336650262]
8,momentum_roc,[1.4421781700222137e-09]
9,volume_obv,[-0.2685556452696494]


In [45]:
y_predict = model.predict(X_test)
metrics.confusion_matrix(y_test, y_predict)
print(metrics.classification_report(y_test, y_predict))

              precision    recall  f1-score   support

          -1       0.65      1.00      0.79        11
           1       0.00      0.00      0.00         6

    accuracy                           0.65        17
   macro avg       0.32      0.50      0.39        17
weighted avg       0.42      0.65      0.51        17



C:\Users\jwnic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jwnic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jwnic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
model.score(X_test, y_test)

0.6470588235294118

### Adding data from Stocktwits

We scraped additional data from Stocktwits, dating back to 12/31/2022. For each day, we count the number of posts labeled "bearish" and the number labeled "bullish". We rerun the regression with normalized features.

In [47]:
df = pd.read_csv('data/AAPL_data.csv')
df.dropna()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2017-04-28,36.075001,35.817501,36.022499,35.912498,83441600.0,33.907143
1,2017-05-01,36.799999,36.240002,36.275002,36.645000,134411600.0,34.598736
2,2017-05-02,37.022499,36.709999,36.884998,36.877499,181408800.0,34.818253
3,2017-05-03,36.872501,36.067501,36.397499,36.764999,182788000.0,34.712040
4,2017-05-04,36.785000,36.452499,36.630001,36.632500,93487600.0,34.586937
...,...,...,...,...,...,...,...
1255,2022-04-22,167.869995,161.500000,166.460007,161.789993,84775200.0,161.789993
1256,2022-04-25,163.169998,158.460007,161.119995,162.880005,96046400.0,162.880005
1257,2022-04-26,162.339996,156.720001,162.250000,156.800003,95623200.0,156.800003
1258,2022-04-27,159.789993,155.380005,155.910004,156.570007,88063200.0,156.570007


In [48]:

df["Date"]=pd.to_datetime(df.Date, format="%Y-%m-%d")
df.index=df['Date']
data = df.sort_index(ascending=True, axis=0)
add_datepart(df, 'Date', drop=False)
df.drop('Elapsed', axis=1, inplace=True)
df

,Date,High,Low,Open,Close,Volume,Adj Close,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start
Date,,,,,,,,,,,,,,,,,,,
2017-04-28,2017-04-28,36.075001,35.817501,36.022499,35.912498,83441600.0,33.907143,2017,4,17,28,4,118,False,False,False,False,False,False
2017-05-01,2017-05-01,36.799999,36.240002,36.275002,36.645000,134411600.0,34.598736,2017,5,18,1,0,121,False,True,False,False,False,False
2017-05-02,2017-05-02,37.022499,36.709999,36.884998,36.877499,181408800.0,34.818253,2017,5,18,2,1,122,False,False,False,False,False,False
2017-05-03,2017-05-03,36.872501,36.067501,36.397499,36.764999,182788000.0,34.712040,2017,5,18,3,2,123,False,False,False,False,False,False
2017-05-04,2017-05-04,36.785000,36.452499,36.630001,36.632500,93487600.0,34.586937,2017,5,18,4,3,124,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-22,2022-04-22,167.869995,161.500000,166.460007,161.789993,84775200.0,161.789993,2022,4,16,22,4,112,False,False,False,False,False,False
2022-04-25,2022-04-25,163.169998,158.460007,161.119995,162.880005,96046400.0,162.880005,2022,4,17,25,0,115,False,False,False,False,False,False
2022-04-26,2022-04-26,162.339996,156.720001,162.250000,156.800003,95623200.0,156.800003,2022,4,17,26,1,116,False,False,False,False,False,False


In [49]:
df = add_all_ta_features(
    df, high="High", low="Low", open="Open", close="Close", volume="Volume")

C:\Users\jwnic\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\jwnic\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [50]:
def convert(date_string):
    year, month, day = [int(i) for i in date_string.split('-')]
    return datetime(year=year, month=month, day=day)

In [51]:
df_sentiment = pd.read_csv('data/AAPL_stocktwits_sentiment.csv')
df_sentiment.date = df_sentiment.date.apply(convert)
df_sentiment.rename(columns={'date':'Date'}, inplace=True)

df = df[df['Date'] >= '2021-12-31']
df.index=np.array(range(len(df)))
selected_features = ['trend_sma_fast', 'trend_ema_fast', 'momentum_stoch_rsi_k', 'momentum_stoch_rsi_d', 'momentum_rsi', \
                    'trend_macd', 'momentum_wr', 'volume_adi', 'momentum_roc', 'volume_obv', \
                    'volatility_bbh', 'volatility_bbl']
basic_features = ['High', 'Low', 'Open', 'Volume', 'Year', 'Month', 'Week', 'Day', 'Dayofweek']
sentiment_features = ['bullish', 'bearish']

df=df.merge(df_sentiment, how='left', on='Date').fillna(0)

X_df = df[selected_features + basic_features + sentiment_features]
X = normalize(X_df.values)
y = np.where(df['Close'].shift(-1) > df['Close'], 1, -1)

split = int(0.8 * len(df))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

model = LogisticRegression(max_iter=float('inf'))
model = model.fit(X_train, y_train)

pd.DataFrame(zip(X_df.columns, np.transpose(model.coef_)))

,0,1
0,trend_sma_fast,[-3.1250300600316886e-09]
1,trend_ema_fast,[-3.0375617739632016e-09]
2,momentum_stoch_rsi_k,[1.1591441459317456e-10]
3,momentum_stoch_rsi_d,[9.601813077212809e-11]
4,momentum_rsi,[2.0732797923957882e-09]
5,trend_macd,[-1.0629743784270782e-09]
6,momentum_wr,[8.401023527239148e-09]
7,volume_adi,[0.13170692369566042]
8,momentum_roc,[1.5474022751503264e-09]
9,volume_obv,[-0.22103197407730346]


In [52]:
y_predict = model.predict(X_test)
metrics.confusion_matrix(y_test, y_predict)
print(metrics.classification_report(y_test, y_predict))

              precision    recall  f1-score   support

          -1       0.65      1.00      0.79        11
           1       0.00      0.00      0.00         6

    accuracy                           0.65        17
   macro avg       0.32      0.50      0.39        17
weighted avg       0.42      0.65      0.51        17



C:\Users\jwnic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jwnic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jwnic\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
model.score(X_test, y_test)

0.6470588235294118

We see that adding the sentiment from Stocktwits did not change the model's test accuracy. This is likely because, as we can see, the model is still determined by a subset of features with comparatively massive coefficients, such as volume_adi, despite the fact that we normalized features.

### Conclusion

Overall, the accuracy of basic logistic regression model was at most 0.59. The model performed best when we included an adequate number of features. We then improved this accuracy when normalizing features to 0.65. Unfortunately, incorporating the sentiment of pre-labeled posts did not improve them model's predictive power, largely because, after feature normalization, a few features have coefficients which are orders of magnitude larger than the other features. We will have to continue to explore feature selection and normalization techniques in order to refine our model. Additionally, to increase the accuracy, we may need to use more sophisticated ML models than Logistic Regression.

followed the tutorial and changed based on dataset:  
https://blog.quantinsti.com/machine-learning-logistic-regression-python/#:~:text=Logistic%20Regression%20and%20Linear%20regression&text=It%20is%20a%20classification%20problem,as%20a%20generalized%20linear%20model.